## Name: Yen-Hsin Fang
## Section: 01

# Lab 8: Exploring Random Forests

## Tools

#### Libraries:

- numpy: for processing
- sklearn: for model training  
- pandas: for data processing  
- rfpimp **version 1.3.7**: for feature importance

#### Datasets:

Boston housing 

## Setup

In [11]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from rfpimp import *

pd.options.mode.chained_assignment = None  # default='warn'

# Ignore all future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from types import SimpleNamespace
def load_boston(return_X_y=False):
    """Replacement function for loading in Boston House Prices"""
    df = pd.read_csv('./data/boston_house_prices.csv')
    X = df.drop(columns=['MEDV'])
    y = df['MEDV'].to_numpy()

    if return_X_y:
        return X, y 
    
    dataset  = SimpleNamespace(data=X, target=y)
    
    return dataset

In [12]:
def boston():
    boston = load_boston()
    df = boston.data
    y = boston.target
    df['y'] = y
    return df

In [13]:
df_boston = boston()
X, y = df_boston.drop('y', axis=1), df_boston['y']
y *= 1000 # y is "Median value of owner-occupied homes in $1000's" so multiply by 1000
X.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33


---
## Train random forests of different sizes in terms of number of trees

In this section we will see that as we increase the number of trees in the ensemble/forest, we should initially see model bias going down, i.e. the predictions getting better. It will asymptotically approach some minimum error on the testing set.

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

Here's how to train a random forest  that has a single tree:

In [15]:
rf = RandomForestRegressor(n_estimators=1)
rf.fit(X_train, y_train)

RandomForestRegressor(n_estimators=1)

---
**Task**: Compute the MAE for the training and the testing set, printing them out.

In [16]:
mae_train = mean_absolute_error(y_train, rf.predict(X_train))
mae = mean_absolute_error(y_test, rf.predict(X_test))
print(f"MAE train {mae_train:.1f}, test {mae:.1f}")

MAE train 1198.3, test 2987.3


---
**Task**: Create a quick loop and run the training and testing cycle above several times to see the variance of the training and testing set errors. You should notice that the test set scores bounce around a lot.

In [18]:
for i in range(5):
    rf = RandomForestRegressor(n_estimators=1)
    rf.fit(X_train, y_train)
    mae_train = mean_absolute_error(y_train, rf.predict(X_train))
    mae = mean_absolute_error(y_test, rf.predict(X_test))
    print(f"MAE train {mae_train:.1f}, test {mae:.1f}")

MAE train 1194.6, test 3290.2
MAE train 1181.7, test 3123.5
MAE train 1127.7, test 2740.2
MAE train 1073.5, test 2866.7
MAE train 1141.3, test 2471.6


---
**Task**: Increase the number of trees (`n_estimators`) to 2, retrain, and print out the results.

In [19]:
for i in range(5):
    rf = RandomForestRegressor(n_estimators=2)
    rf.fit(X_train, y_train)
    mae_train = mean_absolute_error(y_train, rf.predict(X_train))
    mae = mean_absolute_error(y_test, rf.predict(X_test))
    print(f"MAE train {mae_train:.1f}, test {mae:.1f}")

MAE train 1098.8, test 2669.6
MAE train 1207.4, test 2702.9
MAE train 1186.5, test 2495.1
MAE train 1135.6, test 2774.0
MAE train 1227.4, test 2534.3


You should notice that the scores don't bounce around as much as they did when you were only training one tree.

---
**Q.**  Why does the MAE score go down?

Because RF averaging predictions from multiple decision trees. As you increase the number of trees, the variance of the model tends to decrease therefore MAE goes down

---
**Task**: Increase the number of trees (`n_estimators`) to 10, retrain, and print out the results.

In [23]:
for i in range(5):
    rf = RandomForestRegressor(n_estimators=10)
    rf.fit(X_train, y_train) 
    mae_train = mean_absolute_error(y_train, rf.predict(X_train))
    mae = mean_absolute_error(y_test, rf.predict(X_test))
    print(f"MAE train {mae_train:.1f}, test {mae:.1f}")

MAE train 1024.2, test 2076.6
MAE train 932.1, test 2113.3
MAE train 942.6, test 2051.8
MAE train 952.4, test 2132.5
MAE train 978.7, test 2035.9


---
**Q.**  What do you notice about the MAE scores?

It decreases in both training and testing process.

---
**Q.**  After running several times, what else do you notice?

It doesn't bounce that much. But the training time is longer.

---
**Task**: Increase the number of trees (`n_estimators`) to 200, retrain, and print out the results.

In [24]:
for i in range(5):
    rf = RandomForestRegressor(n_estimators=200)
    rf.fit(X_train, y_train) 
    mae_train = mean_absolute_error(y_train, rf.predict(X_train))
    mae = mean_absolute_error(y_test, rf.predict(X_test))
    print(f"MAE train {mae_train:.1f}, test {mae:.1f}")

MAE train 866.9, test 1906.7
MAE train 853.7, test 1890.6
MAE train 845.3, test 1915.7
MAE train 853.1, test 1958.5
MAE train 851.1, test 1931.1


---
**Q.**  What do you notice about the MAE scores from a single run?

MAE drops in training and testing, but not drop significantly.

<details>
<summary>Solution</summary>
Both training and testing error have dropped, but not as significantly as before, even with 200 trees.
</details>

---
**Task**: Notice that it took a little bit longer to train.  Do the exact same thing again but this time use `n_jobs=-1` as an argument to the `RandomForestRegressor` constructor.

This tells the library to use all processing cores available on the computer processor. As long as the data is not too huge (because it must pass it around), it often goes much faster using this argument. It should take less than two seconds.

In [25]:
for i in range(5):
    rf = RandomForestRegressor(n_estimators=200, n_jobs=-1)
    rf.fit(X_train, y_train) 
    mae_train = mean_absolute_error(y_train, rf.predict(X_train))
    mae = mean_absolute_error(y_test, rf.predict(X_test))
    print(f"MAE train {mae_train:.1f}, test {mae:.1f}")

MAE train 854.9, test 1906.7
MAE train 844.7, test 1955.7
MAE train 850.2, test 1923.0
MAE train 856.5, test 1934.5
MAE train 838.7, test 1901.0


---
**Q.**  What do you notice about the MAE scores from SEVERAL runs?

The MAE doesn't bounce that much, having lower varience.

<details>
<summary>Solution</summary>
The error variance across runs is even lower (tighter).
</details>

---
## Examining model size and complexity

The structure of a tree is affected by a number of hyperparameters, not just the data. The goal in this section is to see the effect of altering the number of observations per leaf and the maximum number of candidate features per split. Let's start out with a handy function that uses some  support code from rfpimp to examine tree size and depth:

In [26]:
def showsize(ntrees, max_features=1.0, min_samples_leaf=1):
    rf = RandomForestRegressor(n_estimators=ntrees,
                               max_features=max_features,
                               min_samples_leaf=min_samples_leaf,
                               n_jobs=-1)
    rf.fit(X_train, y_train)
    n = rfnnodes(rf)                # from rfpimp
    h = np.median(rfmaxdepths(rf))  # rfmaxdepths from rfpimp
    mae_train = mean_absolute_error(y_train, rf.predict(X_train))
    mae = mean_absolute_error(y_test, rf.predict(X_test))
    print(f"MAE train {mae_train:6.1f}, test {mae:6.1f} using {n:9,d} tree nodes with {h:2.0f} median tree height")

### Effect of number of trees

For a single tree, we see about 480 nodes and a tree height of around 19:

In [27]:
showsize(ntrees=1)

MAE train 1262.1, test 3202.0 using       477 tree nodes with 16 median tree height


---
**Task**: Look at the metrics for 2 trees and then 100 trees.

In [28]:
showsize(ntrees=2)

MAE train 1242.0, test 2743.6 using       952 tree nodes with 18 median tree height


In [29]:
showsize(ntrees=10)

MAE train  990.6, test 2161.8 using     4,754 tree nodes with 19 median tree height


---
**Q.** Why does the median height of a tree stay the same when we increase the number of trees?

the height of each individual tree remains unchanged since the construction process for a single tree remains constant

<details>
<summary>Solution</summary>
While the number of nodes increases with the number of trees, the height of any individual tree will stay the same because we have not fundamentally changed how it is constructing a single tree.
</details>

### Effect of increasing min samples / leaf

**Task**: Loop around a call to `showsize()` with 10 trees and min_samples_leaf=1..10 

In [35]:
for i in range(1,11):
    print(f"{i:2d} ",end='')
    showsize(10, min_samples_leaf=i)

 1 MAE train  980.9, test 2063.0 using     4,770 tree nodes with 18 median tree height
 2 MAE train 1145.4, test 2180.6 using     2,222 tree nodes with 16 median tree height
 3 MAE train 1382.3, test 2273.6 using     1,422 tree nodes with 14 median tree height
 4 MAE train 1596.0, test 2284.4 using     1,032 tree nodes with 12 median tree height
 5 MAE train 1783.2, test 2548.4 using       808 tree nodes with 12 median tree height
 6 MAE train 1940.5, test 2460.2 using       642 tree nodes with 10 median tree height
 7 MAE train 1940.8, test 2316.3 using       578 tree nodes with 10 median tree height
 8 MAE train 2008.3, test 2531.1 using       500 tree nodes with  9 median tree height
 9 MAE train 2106.1, test 2366.7 using       436 tree nodes with  9 median tree height
10 MAE train 2162.8, test 2498.3 using       386 tree nodes with  8 median tree height


---
**Q.** Why do the median height of a tree and number of total nodes decrease as we increase the number of samples per leaf?

Increasing the number of samples per leaf imposes a constraint that each leaf node must contain at least a certain number of samples. This tends to result in shallower trees

---
**Q.**  Why does the MAE error increase?

Increasing the number of observations in a leaf node results in a more general but less accurate prediction, as the average encompasses a larger and potentially more diverse set of data.

<details>
<summary>Solution</summary>
If we include more observations in a single leaf, then the average is taken over more observations. That average is a more general prediction but less accurate.
</details> 

It's pretty clear from that print out that `min_samples_leaf=1` is the best choice because it gives the minimum validation error.

### Effect of reducing max_features

**Task:** Do another loop from `max_features` = 4 down to 1, with 1 sample per leaf. (There are 4 total features.)

In [34]:
p = X_train.shape[1]
for i in range(p,0,-1):
    print(f"{i:2d} ",end='')
    showsize(ntrees=10, max_features=i)

13 MAE train  960.0, test 2241.7 using     4,752 tree nodes with 20 median tree height
12 MAE train 1005.8, test 2177.2 using     4,750 tree nodes with 20 median tree height
11 MAE train 1063.1, test 2101.2 using     4,762 tree nodes with 19 median tree height
10 MAE train 1010.7, test 2244.3 using     4,800 tree nodes with 20 median tree height
 9 MAE train  998.1, test 1974.8 using     4,784 tree nodes with 20 median tree height
 8 MAE train  914.7, test 2114.3 using     4,886 tree nodes with 19 median tree height
 7 MAE train  960.6, test 1950.7 using     4,878 tree nodes with 18 median tree height
 6 MAE train 1006.3, test 2143.4 using     4,784 tree nodes with 19 median tree height
 5 MAE train  918.7, test 2020.7 using     4,904 tree nodes with 19 median tree height
 4 MAE train 1006.7, test 2352.8 using     4,900 tree nodes with 19 median tree height
 3 MAE train 1059.3, test 2535.7 using     4,946 tree nodes with 20 median tree height
 2 MAE train 1096.9, test 2504.1 using     

For this data set, changing the number of candidate features does not change the height of the tree much nor do we see a very clear pattern for the test set error, e.g. the error clearly increasing or decreasing as we decrease the number of candidates. 

## RF prediction confidence

A random forest is a collection of decision trees, each of which contributes a prediction. The forest averages those predictions to provide the overall prediction (or takes most common vote for classification). Let's dig inside the random forest to get the individual trees out and ask them what their predictions are.

**Task**: Train a random forest with 10 trees on `X_train`, `y_train`.  Use `for t in rf.estimators_` to iterate through the trees making predictions with `t` not `rf`. Print out the usual MAE scores for each tree predictor.

In [36]:
rf = RandomForestRegressor(n_estimators=10, n_jobs=-1)
rf.fit(X_train, y_train)

for t in rf.estimators_:
    mae_train = mean_absolute_error(y_train, t.predict(X_train))
    mae = mean_absolute_error(y_test, t.predict(X_test))
    print(f"MAE train {mae_train:.1f}, test {mae:.1f}")

MAE train 1276.2, test 3532.4
MAE train 1130.0, test 2621.6
MAE train 1330.2, test 2960.8
MAE train 1370.0, test 3183.3
MAE train 1208.2, test 2532.4
MAE train 1333.7, test 3253.9
MAE train 1101.0, test 3035.3
MAE train 1402.2, test 3110.8
MAE train 1286.6, test 3148.0
MAE train 1186.4, test 2810.8


/opt/homebrew/anaconda3/envs/ml/lib/python3.11/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(
/opt/homebrew/anaconda3/envs/ml/lib/python3.11/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(
/opt/homebrew/anaconda3/envs/ml/lib/python3.11/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(
/opt/homebrew/anaconda3/envs/ml/lib/python3.11/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(
/opt/homebrew/anaconda3/envs/ml/lib/python3.11/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(
/opt/homebrew/anaconda3/envs/ml/lib/pyth

Notice that it bounces around quite a bit. 

---
**Task**: Select any one of the `X_test` rows and print out the predicted rent price.

In [37]:
x = X_test.iloc[30,:] # pick single test case
x = x.values.reshape(1,-1)
print(f"{x} => {rf.predict(x)}")

[[  0.537    0.       6.2      0.       0.504    5.981   68.1      3.6715
    8.     307.      17.4    378.35    11.65  ]] => [21150.]


/opt/homebrew/anaconda3/envs/ml/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


---
**Task**: Now let's see how the forest came to that conclusion. Compute the average of the predictions obtained from every tree. Compare that to the prediction obtained directly from the random forest (`rf.predict(X_test)`). They should be the same.

In [39]:
y_pred = np.mean([t.predict(x) for t in rf.estimators_])
print(f"{x} => {y_pred}")

[[  0.537    0.       6.2      0.       0.504    5.981   68.1      3.6715
    8.     307.      17.4    378.35    11.65  ]] => 21150.0


<details>
<summary>Solution</summary>
<pre>
y_pred = np.mean([t.predict(x) for t in rf.estimators_])
print(f"{x} => {y_pred}$")
</pre>
</details>

---
**Task**: Compute the standard deviation of the tree estimates and print that out.

In [40]:
np.std([t.predict(x) for t in rf.estimators_])

2542.538101976055

<details>
<summary>Solution</summary>
<pre>
np.std([t.predict(x) for t in rf.estimators_])
</pre>
</details>

The lower the standard deviation, the more tightly grouped the predictions were, which means we should have more confidence in our answer. 

Different records will often have different standard deviations, which means we could have different levels of confidence in the various answers. This might be helpful to a bank for example that wanted to not only predict whether to give loans, but how confident the model was.

## Altering bootstrap size

In this section we will tune one final hyperparameter and see how it affects the model: the `max_samples` which controls the size of the bootstrapped data set used for fitting each tree.

**Task**: There are only about 400 training records, change that to 200 and check the error again.

In [41]:
rf = RandomForestRegressor(n_estimators=200) # don't compute in parallel so we can see timing
%time rf.fit(X_train, y_train)
mae_train = mean_absolute_error(y_train, rf.predict(X_train))
mae = mean_absolute_error(y_test, rf.predict(X_test))
print(f"MAE train {mae_train:.1f}, test {mae:.1f}")

CPU times: user 318 ms, sys: 3.19 ms, total: 321 ms
Wall time: 325 ms
MAE train 848.3, test 1917.1


In [42]:
rf = RandomForestRegressor(n_estimators=200, max_samples=1/2)
%time rf.fit(X_train, y_train)
mae_train = mean_absolute_error(y_train, rf.predict(X_train))
mae = mean_absolute_error(y_test, rf.predict(X_test))
print(f"MAE train {mae_train:.1f}, test {mae:.1f}")

CPU times: user 209 ms, sys: 1.96 ms, total: 211 ms
Wall time: 211 ms
MAE train 1428.7, test 2026.6


It's a bit less accurate, but it's faster.

---
**Q.**  Why is it less accurate?

While this randomness can enhance the overall performance of the Random Forest, it may also lead to less accurate predictions for individual trees.

---
**Task**: Turn off bootstrapping by adding `bootstrap=False` to the constructor of the model. This means that it will subsample rather than bootstrap. Remember that bootstrapping gets about two thirds of the data because of replacement.

In [43]:
rf = RandomForestRegressor(n_estimators=200, n_jobs=-1, bootstrap=False)
%time rf.fit(X_train, y_train)
mae_train = mean_absolute_error(y_train, rf.predict(X_train))
mae = mean_absolute_error(y_test, rf.predict(X_test))
print(f"MAE train {mae_train:.1f}$, test {mae:.1f}$")

CPU times: user 547 ms, sys: 60.3 ms, total: 607 ms
Wall time: 149 ms
MAE train 0.0$, test 2873.0$


Notice what happened to the training set error. It got quite a bit lower when we do not do bootstrapping, we are overfitting by a lot.